# 1. Load libraries and data

In [140]:
#!pwd

In [141]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import timedelta
import os
pd.options.mode.chained_assignment = None 

## Append csv files

In [142]:
def add_dataframes():
    df = pd.DataFrame()
    for file_name in os.listdir('../data/'):
        df_weekday = pd.read_csv('../data/'+file_name, delimiter=";")
        #########
        df=pd.concat([df,df_weekday],axis=0).reset_index(drop=True)
    df=df.sort_values(['timestamp'],axis = 0, ascending = True).reset_index(drop=True)
    return df

In [143]:
df_raw=add_dataframes()
df_raw

,timestamp,customer_no,location
0,2019-09-02 07:03:00,1,dairy
1,2019-09-02 07:03:00,2,dairy
2,2019-09-02 07:04:00,4,dairy
3,2019-09-02 07:04:00,5,spices
4,2019-09-02 07:04:00,6,spices
...,...,...,...
24872,2019-09-06 21:50:00,1509,drinks
24873,2019-09-06 21:50:00,1507,checkout
24874,2019-09-06 21:50:00,1496,fruit
24875,2019-09-06 21:50:00,1508,checkout


# 2. Inspect Data

In [144]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24877 entries, 0 to 24876
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   timestamp    24877 non-null  object
 1   customer_no  24877 non-null  int64 
 2   location     24877 non-null  object
dtypes: int64(1), object(2)
memory usage: 583.2+ KB


In [145]:
#convert column timestamp to datetime
df_raw['timestamp'] = pd.to_datetime(df_raw['timestamp'])
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24877 entries, 0 to 24876
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   timestamp    24877 non-null  datetime64[ns]
 1   customer_no  24877 non-null  int64         
 2   location     24877 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 583.2+ KB


## create df missing checkouts

In [146]:
# number of state checkout
customer_checkout=df_raw['location'].value_counts()[0]
customer_checkout

7417

In [147]:
# number of all customers in the week
customer_total=df_raw.groupby([df_raw['timestamp'].dt.date, 'customer_no']).first().shape[0]
customer_total

7445

In [148]:
#customers without checkout timestemp
not_checkout=customer_total-customer_checkout
not_checkout

28

In [149]:
#create column with last recorded timestamp for each customer per day (grouped by day+customer_no)
df_raw['timestamp_last'] = df_raw.groupby([df_raw['timestamp'].dt.date, df_raw['customer_no']])['timestamp'].transform('last')
df_raw

,timestamp,customer_no,location,timestamp_last
0,2019-09-02 07:03:00,1,dairy,2019-09-02 07:05:00
1,2019-09-02 07:03:00,2,dairy,2019-09-02 07:06:00
2,2019-09-02 07:04:00,4,dairy,2019-09-02 07:08:00
3,2019-09-02 07:04:00,5,spices,2019-09-02 07:05:00
4,2019-09-02 07:04:00,6,spices,2019-09-02 07:29:00
...,...,...,...,...
24872,2019-09-06 21:50:00,1509,drinks,2019-09-06 21:50:00
24873,2019-09-06 21:50:00,1507,checkout,2019-09-06 21:50:00
24874,2019-09-06 21:50:00,1496,fruit,2019-09-06 21:50:00
24875,2019-09-06 21:50:00,1508,checkout,2019-09-06 21:50:00


In [150]:
df_missing_checkouts = df_raw.loc[(df_raw['timestamp'] == df_raw['timestamp_last']) & (df_raw['location'] != 'checkout')]
df_missing_checkouts

,timestamp,customer_no,location,timestamp_last
4857,2019-09-02 21:46:00,1439,fruit,2019-09-02 21:46:00
4865,2019-09-02 21:47:00,1437,dairy,2019-09-02 21:47:00
4873,2019-09-02 21:48:00,1443,dairy,2019-09-02 21:48:00
4875,2019-09-02 21:49:00,1445,dairy,2019-09-02 21:49:00
4878,2019-09-02 21:49:00,1430,fruit,2019-09-02 21:49:00
4879,2019-09-02 21:49:00,1440,spices,2019-09-02 21:49:00
4880,2019-09-02 21:49:00,1433,fruit,2019-09-02 21:49:00
4881,2019-09-02 21:49:00,1441,spices,2019-09-02 21:49:00
4882,2019-09-02 21:50:00,1446,dairy,2019-09-02 21:50:00
4883,2019-09-02 21:50:00,1447,fruit,2019-09-02 21:50:00


## create df missing entries

In [151]:
#create column with last recorded timestamp for each customer per day (grouped by day+customer_no)
df_raw['timestamp_first'] = df_raw.groupby([df_raw['timestamp'].dt.date, df_raw['customer_no']])['timestamp'].transform('first')
df_raw

,timestamp,customer_no,location,timestamp_last,timestamp_first
0,2019-09-02 07:03:00,1,dairy,2019-09-02 07:05:00,2019-09-02 07:03:00
1,2019-09-02 07:03:00,2,dairy,2019-09-02 07:06:00,2019-09-02 07:03:00
2,2019-09-02 07:04:00,4,dairy,2019-09-02 07:08:00,2019-09-02 07:04:00
3,2019-09-02 07:04:00,5,spices,2019-09-02 07:05:00,2019-09-02 07:04:00
4,2019-09-02 07:04:00,6,spices,2019-09-02 07:29:00,2019-09-02 07:04:00
...,...,...,...,...,...
24872,2019-09-06 21:50:00,1509,drinks,2019-09-06 21:50:00,2019-09-06 21:50:00
24873,2019-09-06 21:50:00,1507,checkout,2019-09-06 21:50:00,2019-09-06 21:48:00
24874,2019-09-06 21:50:00,1496,fruit,2019-09-06 21:50:00,2019-09-06 21:35:00
24875,2019-09-06 21:50:00,1508,checkout,2019-09-06 21:50:00,2019-09-06 21:48:00


In [152]:
df_missing_entries = df_raw.loc[(df_raw['timestamp'] == df_raw['timestamp_first'])]
df_missing_entries

,timestamp,customer_no,location,timestamp_last,timestamp_first
0,2019-09-02 07:03:00,1,dairy,2019-09-02 07:05:00,2019-09-02 07:03:00
1,2019-09-02 07:03:00,2,dairy,2019-09-02 07:06:00,2019-09-02 07:03:00
2,2019-09-02 07:04:00,4,dairy,2019-09-02 07:08:00,2019-09-02 07:04:00
3,2019-09-02 07:04:00,5,spices,2019-09-02 07:05:00,2019-09-02 07:04:00
4,2019-09-02 07:04:00,6,spices,2019-09-02 07:29:00,2019-09-02 07:04:00
...,...,...,...,...,...
24864,2019-09-06 21:48:00,1506,dairy,2019-09-06 21:48:00,2019-09-06 21:48:00
24865,2019-09-06 21:48:00,1507,dairy,2019-09-06 21:50:00,2019-09-06 21:48:00
24866,2019-09-06 21:48:00,1508,dairy,2019-09-06 21:50:00,2019-09-06 21:48:00
24871,2019-09-06 21:50:00,1510,spices,2019-09-06 21:50:00,2019-09-06 21:50:00


In [153]:
#all customers have no Entry timestep?
df_missing_entries.shape[0]==customer_total

True

# 3. Preprocessing

## 3.1. Missing checkouts/entries

In [154]:
def insert_rows(df_original,df_checkout,df_entry):
    #Name location to checkout
    df_checkout.loc[:,['location']] = ['checkout']  
    #change time-stamp higher
    df_checkout['timestamp'] = df_checkout['timestamp'] + pd.Timedelta(minutes=1)
    
    #Name location to entry
    df_entry.loc[:,['location']] = ['entry']  
    #change time-stamp lower
    df_entry['timestamp'] = df_entry['timestamp'] + pd.Timedelta(minutes=-1)
    
    #concat to original
    df_new=pd.concat([df_original,df_checkout,df_entry],axis=0)
    
    print(f' Sum rows of all df`s is equal to rows df_clean: {df_original.shape[0]+df_checkout.shape[0]+df_entry.shape[0]==df_new.shape[0]}')
    
    return df_new

In [155]:
# insert missing checkout rows
df_clean=insert_rows(df_raw,df_missing_checkouts,df_missing_entries)
df_clean

 Sum rows of all df`s is equal to rows df_clean: True


,timestamp,customer_no,location,timestamp_last,timestamp_first
0,2019-09-02 07:03:00,1,dairy,2019-09-02 07:05:00,2019-09-02 07:03:00
1,2019-09-02 07:03:00,2,dairy,2019-09-02 07:06:00,2019-09-02 07:03:00
2,2019-09-02 07:04:00,4,dairy,2019-09-02 07:08:00,2019-09-02 07:04:00
3,2019-09-02 07:04:00,5,spices,2019-09-02 07:05:00,2019-09-02 07:04:00
4,2019-09-02 07:04:00,6,spices,2019-09-02 07:29:00,2019-09-02 07:04:00
...,...,...,...,...,...
24864,2019-09-06 21:47:00,1506,entry,2019-09-06 21:48:00,2019-09-06 21:48:00
24865,2019-09-06 21:47:00,1507,entry,2019-09-06 21:50:00,2019-09-06 21:48:00
24866,2019-09-06 21:47:00,1508,entry,2019-09-06 21:50:00,2019-09-06 21:48:00
24871,2019-09-06 21:49:00,1510,entry,2019-09-06 21:50:00,2019-09-06 21:50:00


In [159]:
#df_clean['shifty']=df_clean.groupby([df_clean['timestamp'].dt.date,'customer_no'])[['location']].count()
df_clean.loc[df_clean['customer_no'] == 1].sort_values('timestamp')

,timestamp,customer_no,location,timestamp_last,timestamp_first
0,2019-09-02 07:02:00,1,entry,2019-09-02 07:05:00,2019-09-02 07:03:00
0,2019-09-02 07:03:00,1,dairy,2019-09-02 07:05:00,2019-09-02 07:03:00
8,2019-09-02 07:05:00,1,checkout,2019-09-02 07:05:00,2019-09-02 07:03:00
4884,2019-09-03 07:01:00,1,entry,2019-09-03 07:12:00,2019-09-03 07:02:00
4884,2019-09-03 07:02:00,1,fruit,2019-09-03 07:12:00,2019-09-03 07:02:00
4887,2019-09-03 07:05:00,1,drinks,2019-09-03 07:12:00,2019-09-03 07:02:00
4909,2019-09-03 07:12:00,1,checkout,2019-09-03 07:12:00,2019-09-03 07:02:00
9598,2019-09-04 06:59:00,1,entry,2019-09-04 07:02:00,2019-09-04 07:00:00
9598,2019-09-04 07:00:00,1,fruit,2019-09-04 07:02:00,2019-09-04 07:00:00
9607,2019-09-04 07:02:00,1,checkout,2019-09-04 07:02:00,2019-09-04 07:00:00


## 3.2. Forward fill every minute

In [178]:
#Create date (yyyy-mm-dd) column
df_clean['date']=df_clean['timestamp'].dt.date
df_clean

,timestamp,customer_no,location,timestamp_last,timestamp_first,date
0,2019-09-02 07:03:00,1,dairy,2019-09-02 07:05:00,2019-09-02 07:03:00,2019-09-02
1,2019-09-02 07:03:00,2,dairy,2019-09-02 07:06:00,2019-09-02 07:03:00,2019-09-02
2,2019-09-02 07:04:00,4,dairy,2019-09-02 07:08:00,2019-09-02 07:04:00,2019-09-02
3,2019-09-02 07:04:00,5,spices,2019-09-02 07:05:00,2019-09-02 07:04:00,2019-09-02
4,2019-09-02 07:04:00,6,spices,2019-09-02 07:29:00,2019-09-02 07:04:00,2019-09-02
...,...,...,...,...,...,...
24864,2019-09-06 21:47:00,1506,entry,2019-09-06 21:48:00,2019-09-06 21:48:00,2019-09-06
24865,2019-09-06 21:47:00,1507,entry,2019-09-06 21:50:00,2019-09-06 21:48:00,2019-09-06
24866,2019-09-06 21:47:00,1508,entry,2019-09-06 21:50:00,2019-09-06 21:48:00,2019-09-06
24871,2019-09-06 21:49:00,1510,entry,2019-09-06 21:50:00,2019-09-06 21:50:00,2019-09-06


In [179]:
#Set index to timestamp
df_test=df_clean.set_index('timestamp').sort_index()
df_test
#df_test.loc[df_test['customer_no'] == 1] # .sort_values('timestamp')

,customer_no,location,timestamp_last,timestamp_first,date
timestamp,,,,,
2019-09-02 07:02:00,1,entry,2019-09-02 07:05:00,2019-09-02 07:03:00,2019-09-02
2019-09-02 07:02:00,2,entry,2019-09-02 07:06:00,2019-09-02 07:03:00,2019-09-02
2019-09-02 07:03:00,4,entry,2019-09-02 07:08:00,2019-09-02 07:04:00,2019-09-02
2019-09-02 07:03:00,5,entry,2019-09-02 07:05:00,2019-09-02 07:04:00,2019-09-02
2019-09-02 07:03:00,6,entry,2019-09-02 07:29:00,2019-09-02 07:04:00,2019-09-02
...,...,...,...,...,...
2019-09-06 21:50:00,1508,checkout,2019-09-06 21:50:00,2019-09-06 21:48:00,2019-09-06
2019-09-06 21:51:00,1510,checkout,2019-09-06 21:50:00,NaT,2019-09-06
2019-09-06 21:51:00,1509,checkout,2019-09-06 21:50:00,NaT,2019-09-06


In [ ]:
list(df_test.groupby(['customer_no','date']))

In [193]:
# Forward fill missing timesteps (every minute is necessary)
# resample gets applied to each dataframe inside the groupby
df_resampled = df_test.groupby(['customer_no','date']).resample('60S').first().ffill()
df_resampled

customer_no  location  \
customer_no date       timestamp                                    
1           2019-09-02 2019-09-02 07:02:00          1.0     entry   
                       2019-09-02 07:03:00          1.0     dairy   
                       2019-09-02 07:04:00          1.0     dairy   
                       2019-09-02 07:05:00          1.0  checkout   
            2019-09-03 2019-09-03 07:01:00          1.0     entry   
...                                                 ...       ...   
1534        2019-09-05 2019-09-05 21:48:00       1534.0     fruit   
                       2019-09-05 21:49:00       1534.0  checkout   
1535        2019-09-05 2019-09-05 21:47:00       1535.0     entry   
                       2019-09-05 21:48:00       1535.0    spices   
                       2019-09-05 21:49:00       1535.0  checkout   

                                                timestamp_last  \
customer_no date       timestamp                                 
1           2019-09-02 2019-09-02 07:02:00 2019-09-02 07:05:00   
                       2019-09-02 07:03:00 2019-09-02 07:05:00   
                       2019-09-02 07:04:00 2019-09-02 07:05:00   
                       2019-09-02 07:05:00 2019-09-02 07:05:00   
            2019-09-03 2019-09-03 07:01:00 2019-09-03 07:12:00   
...                                                        ...   
1534        2019-09-05 2019-09-05 21:48:00 2019-09-05 21:49:00   
                       2019-09-05 21:49:00 2019-09-05 21:49:00   
1535        2019-09-05 2019-09-05 21:47:00 2019-09-05 21:49:00   
                       2019-09-05 21:48:00 2019-09-05 21:49:00   
                       2019-09-05 21:49:00 2019-09-05 21:49:00   

                                               timestamp_first        date  
customer_no date       timestamp                                            
1           2019-09-02 2019-09-02 07:02:00 2019-09-02 07:03:00  2019-09-02  
                       2019-09-02 07:03:00 2019-09-02 07:03:00  2019-09-02  
                       2019-09-02 07:04:00 2019-09-02 07:03:00  2019-09-02  
                       2019-09-02 07:05:00 2019-09-02 07:03:00  2019-09-02  
            2019-09-03 2019-09-03 07:01:00 2019-09-03 07:02:00  2019-09-03  
...                                                        ...         ...  
1534        2019-09-05 2019-09-05 21:48:00 2019-09-05 21:48:00  2019-09-05  
                       2019-09-05 21:49:00 2019-09-05 21:48:00  2019-09-05  
1535        2019-09-05 2019-09-05 21:47:00 2019-09-05 21:48:00  2019-09-05  
                       2019-09-05 21:48:00 2019-09-05 21:48:00  2019-09-05  
                       2019-09-05 21:49:00 2019-09-05 21:48:00  2019-09-05  

[61376 rows x 5 columns]

 # 4. EDA

# 5. Markov Chain Model

## 5.1. Shift

In [194]:
#Creat shifted column
df_resampled['location_next'] = df_resampled['location'].shift(-1)
df_resampled

customer_no  location  \
customer_no date       timestamp                                    
1           2019-09-02 2019-09-02 07:02:00          1.0     entry   
                       2019-09-02 07:03:00          1.0     dairy   
                       2019-09-02 07:04:00          1.0     dairy   
                       2019-09-02 07:05:00          1.0  checkout   
            2019-09-03 2019-09-03 07:01:00          1.0     entry   
...                                                 ...       ...   
1534        2019-09-05 2019-09-05 21:48:00       1534.0     fruit   
                       2019-09-05 21:49:00       1534.0  checkout   
1535        2019-09-05 2019-09-05 21:47:00       1535.0     entry   
                       2019-09-05 21:48:00       1535.0    spices   
                       2019-09-05 21:49:00       1535.0  checkout   

                                                timestamp_last  \
customer_no date       timestamp                                 
1           2019-09-02 2019-09-02 07:02:00 2019-09-02 07:05:00   
                       2019-09-02 07:03:00 2019-09-02 07:05:00   
                       2019-09-02 07:04:00 2019-09-02 07:05:00   
                       2019-09-02 07:05:00 2019-09-02 07:05:00   
            2019-09-03 2019-09-03 07:01:00 2019-09-03 07:12:00   
...                                                        ...   
1534        2019-09-05 2019-09-05 21:48:00 2019-09-05 21:49:00   
                       2019-09-05 21:49:00 2019-09-05 21:49:00   
1535        2019-09-05 2019-09-05 21:47:00 2019-09-05 21:49:00   
                       2019-09-05 21:48:00 2019-09-05 21:49:00   
                       2019-09-05 21:49:00 2019-09-05 21:49:00   

                                               timestamp_first        date  \
customer_no date       timestamp                                             
1           2019-09-02 2019-09-02 07:02:00 2019-09-02 07:03:00  2019-09-02   
                       2019-09-02 07:03:00 2019-09-02 07:03:00  2019-09-02   
                       2019-09-02 07:04:00 2019-09-02 07:03:00  2019-09-02   
                       2019-09-02 07:05:00 2019-09-02 07:03:00  2019-09-02   
            2019-09-03 2019-09-03 07:01:00 2019-09-03 07:02:00  2019-09-03   
...                                                        ...         ...   
1534        2019-09-05 2019-09-05 21:48:00 2019-09-05 21:48:00  2019-09-05   
                       2019-09-05 21:49:00 2019-09-05 21:48:00  2019-09-05   
1535        2019-09-05 2019-09-05 21:47:00 2019-09-05 21:48:00  2019-09-05   
                       2019-09-05 21:48:00 2019-09-05 21:48:00  2019-09-05   
                       2019-09-05 21:49:00 2019-09-05 21:48:00  2019-09-05   

                                           location_next  
customer_no date       timestamp                          
1           2019-09-02 2019-09-02 07:02:00         dairy  
                       2019-09-02 07:03:00         dairy  
                       2019-09-02 07:04:00      checkout  
                       2019-09-02 07:05:00         entry  
            2019-09-03 2019-09-03 07:01:00         fruit  
...                                                  ...  
1534        2019-09-05 2019-09-05 21:48:00      checkout  
                       2019-09-05 21:49:00         entry  
1535        2019-09-05 2019-09-05 21:47:00        spices  
                       2019-09-05 21:48:00      checkout  
                       2019-09-05 21:49:00           NaN  

[61376 rows x 6 columns]

In [196]:
#avoid checkout --> entry pairs
#instead self-reference checkout-->checkout
df_resampled['location_next']
#df_resampled[df_resampled['location_next']=='entry'] = 'checkout'
#df_resampled

customer_no  date        timestamp          
1            2019-09-02  2019-09-02 07:02:00       dairy
                         2019-09-02 07:03:00       dairy
                         2019-09-02 07:04:00    checkout
                         2019-09-02 07:05:00       entry
             2019-09-03  2019-09-03 07:01:00       fruit
                                                  ...   
1534         2019-09-05  2019-09-05 21:48:00    checkout
                         2019-09-05 21:49:00       entry
1535         2019-09-05  2019-09-05 21:47:00      spices
                         2019-09-05 21:48:00    checkout
                         2019-09-05 21:49:00         NaN
Name: location_next, Length: 61376, dtype: object